In [60]:
! pip install scikeras

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.10.0 which is incompatible.



  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
Using cached keras-3.10.0-py3-none-any.whl (1.4 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0


In [61]:
! pip install keras

In [62]:
! pip install tensorflow


  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.10.0
    Uninstalling keras-3.10.0:
      Successfully uninstalled keras-3.10.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.13.0 requires keras>=3.2.0, but you have keras 2.15.0 which is incompatible.


In [63]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import pickle

In [64]:
data = pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df], axis = 1)

X = data.drop('Exited', axis = 1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [65]:
# Save the encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)
    
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo,file)

    
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)

In [66]:
# Define a function to create the model and try different parameters(kerasClassifier)
from tensorflow.keras.layers import Dense

def create_model(neurons = 32, layers = 1):
     
     model = Sequential()
     model.add(Dense(neurons, activation = 'relu', input_shape = (X_train.shape[1],)))
     
     for _ in range(layers-1):
          model.add(Dense(neurons, activation = 'relu'))
          
     model.add(Dense(1, activation = 'sigmoid'))
     model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ['accuracy'])
     
     return model



In [67]:
model = KerasClassifier(layer=1, neurons = 32, build_fn=create_model, verbose=1)


In [68]:
# Define the grid search parameters

param_grid = {
     'neurons' : [16, 32, 64, 128],
     'layers' : [1,2,3],
     'batch_size' : [10,20],
     'epochs' : [50, 100]
}

In [71]:
# # 4. Run Grid Search
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1)
# grid_result = grid.fit(X_train, y_train)

# # 5. Output best result
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))